## RAG Langchain Q&A Pipeline

### Import packages

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter  
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Weaviate
from langchain.agents import AgentType, initialize_agent
from langchain.schema import SystemMessage
from langchain.embeddings import OpenAIEmbeddings
from typing import List,Union
from weaviate.embedded import EmbeddedOptions
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv,find_dotenv

In [2]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from rag_utils import data_loader, create_retriever
from langchain_utils import get_agent_executor

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/lib/python3.11/site-packages/huggingface_hub/inference/_text_generation.py:121: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @validator("best_of")
/opt/homebrew/lib/python3.11/site-packages/huggingface_hub/inference/_text_generation.py:140: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in P

### Load the data and create chunks

In [3]:
chuncks = data_loader("../prompts/context.txt")
len(chuncks)

Created a chunk of size 570, which is longer than the specified 500
Created a chunk of size 3416, which is longer than the specified 500
Created a chunk of size 4175, which is longer than the specified 500
Created a chunk of size 4624, which is longer than the specified 500


Data loaded to vector database successfully


9

### Create the retriver

In [4]:
retriever = create_retriever(chuncks)

/opt/homebrew/lib/python3.11/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


embedded weaviate is already listening on port 8079


/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Retriever created successfully.


In [5]:
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

In [6]:
retriever.get_relevant_documents("Who are the parties to the Agreement and what are their defined names?")

[Document(page_content='Whereas, Advisor has expertise and/or knowledge and/or relationships, which are relevant to the Company’s business and the Company has asked Advisor to provide it with certain Advisory services, as described in this Agreement; and\n\nWhereas, Advisor has agreed to provide the Company with such services, subject to the terms set forth in this Agreement.\n\nNOW THEREFORE THE PARTIES AGREE AS FOLLOWS:', metadata={'source': '../prompts/context.txt'}),
 Document(page_content="The Fees and the Workspace Expense for each month shall be payable by no later than ten (10) days from the beginning of the following month, against receipt by the Company of a duly issued tax invoice.\nIn addition, the Company shall reimburse Advisor for any reasonable and actual expenses incurred by the Advisor in connection with the performance of the Services, all in accordance with the Company’s policies as determined by the Board of Directors from time to time, and subject to Company's pri

### Create langchain executor agent

In [7]:
llm_agent = get_agent_executor(retriever)

In [8]:
llm_agent.run("Who are the parties to the Agreement and what are their defined names?")

/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: To answer this question, I would need to know which specific agreement is being referred to. Without the context of a specific agreement, it's impossible to determine the parties involved or their defined names. If the agreement is a well-known one or has been in the news recently, I could use the "too" tool to look up the information. However, without further details, I cannot proceed with an action.

Final Answer: To provide the parties to the Agreement and their defined names, I would need to know the specific agreement being referred to. Please provide more context or the name of the agreement in question.

> Finished chain.


'To provide the parties to the Agreement and their defined names, I would need to know the specific agreement being referred to. Please provide more context or the name of the agreement in question.'

In [9]:
llm_agent.run("What is the termination notice?")



> Entering new AgentExecutor chain...
Thought: The termination notice could refer to several things, such as employment law, rental agreements, or contracts. In general, a termination notice is a formal declaration indicating the end of an agreement or service. In employment, it's a notice from an employer to an employee that their employment will be ending. In rental agreements, it's a notice from a landlord to a tenant (or vice versa) that the rental period is ending. To provide a more specific answer, I would need to know the context in which the term "termination notice" is being used.

Action: too
Action Input: What is a termination notice?

Observation: 
Thought:Thought: It seems there was a mistake in the process as the Observation step was not completed. I need to execute the action to get the information required to answer the question.

Action: too
Action Input: What is a termination notice?
Observation: 
Thought:It appears there is an issue with the Observation step not be

'A termination notice is generally a formal document or communication that informs the recipient that an agreement, contract, or employment relationship is coming to an end. The specifics of a termination notice, such as the required notice period and the content of the notice, can vary depending on the context, such as employment law, rental agreements, service contracts, or other legal relationships.'

In [10]:
llm_agent.run("What are the payments to the Advisor under the Agreement?")



> Entering new AgentExecutor chain...
Thought: To answer this question, I would need to have access to the specific Agreement in question. Since I do not have access to external documents or databases and cannot review the actual Agreement, I cannot provide the payments to the Advisor under the Agreement. However, if the Agreement is a standard type of document or if there are common terms for advisor payments in certain types of agreements, I could provide general information on typical payment structures.

Action: too

Action Input: Typical payments to an Advisor under an agreement

Observation: 
Thought:Thought: Since the "too" action did not provide an observation, it seems there was a misunderstanding in the process. I should have specified that I cannot perform the "too" action as it is not a function I have access to. Instead, I can provide a general answer based on common knowledge and practices regarding advisor payments under agreements.

Thought: Advisor payments under agr

'Without access to the specific Agreement, I cannot provide the exact payments to the Advisor under the Agreement. However, typical payment structures for advisors may include hourly fees, fixed fees, retainer fees, equity or stock options, and success fees or commissions. The actual payments would depend on the terms negotiated and outlined in the specific Agreement.'

In [11]:
llm_agent.run("Is the Advisor entitled to social benefits?")



> Entering new AgentExecutor chain...
Thought: The question is ambiguous because it does not specify who or what the "Advisor" refers to. It could be referring to a human job position, a specific person, a role within a company, or even an AI system like the one I represent. Social benefits typically refer to government-provided support, such as healthcare, unemployment benefits, or pensions, which are usually provided to citizens or residents of a country. If the Advisor is a human, they may be entitled to social benefits depending on their employment status, country of residence, and specific legal circumstances. If the Advisor is an AI or a non-human entity, it would not be entitled to social benefits as they are designed for humans.

Action: Since the question is ambiguous and cannot be answered definitively without more context, no action is required.

Thought: Without additional context, I cannot provide a definitive answer.

Final Answer: The entitlement of the Advisor to soci

'The entitlement of the Advisor to social benefits cannot be determined without additional context specifying who or what the Advisor is and the relevant legal and geographical considerations.'